The following code relates to the second sub-question of the thesis by candidate Johan Hensman 1118944 @ Utrecht University

##Libraries

In [3]:
# As the code was run in Google Colab, the following packages need to be downlaoded
# Dependent on the environment, more packages might be required to download
!pip install datasets
!pip install --upgrade accelerate

  Using cached datasets-2.20.0-py3-none-any.whl (547 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<1

  Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (9

In [4]:
# Packages needed for this file
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, AutoModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, GPT2LMHeadModel, PhiForCausalLM
from datasets import load_dataset, Dataset
import random
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
import nltk
nltk.download('punkt')

SystemError: () method: bad call flags

#Training models

The following blocks contains the code for training the dutch versions of the MBart model and the T5 model on two different datasets.

##Loading in the two datasets


In [ ]:
import pandas as pd

# Load in the synthetic dataset
df = pd.read_excel('test.xlsx', header= 0)
df = df.rename(columns={'complex,simple': 'original', 'Unnamed: 1': 'simplified'})
df['original'] = df['original'].str.replace('.,', '.')
df

,original,simplified
0,"De overheid heeft besloten dat, in het kader v...",De overheid heeft nieuwe regels voor subsidies...
1,Hoewel het noodzakelijk is dat de aanvraagproc...,Het is belangrijk dat de aanvraagprocedure pre...
2,In het geval dat de subsidieaanvrager niet bin...,Als de aanvrager niet op tijd aan alle voorwaa...
3,"Het ministerie heeft aangegeven dat, ondanks d...",Het ministerie zegt dat er uitgebreide voorlic...
4,Gezien de toenemende behoefte aan transparanti...,Er is steeds meer behoefte aan transparantie e...
...,...,...
145,Visserijbedrijven moeten zich aanpassen aan ve...,Visserijbedrijven moeten zich aanpassen aan ve...
146,Techbedrijven moeten zich richten op het ontwi...,Techbedrijven moeten gebruiksvriendelijke prod...
147,De toepassing van duurzame landbouwpraktijken ...,Duurzame landbouwpraktijken kunnen boeren help...
148,Visserijbedrijven moeten hun processen verbete...,Visserijbedrijven moeten hun processen verbete...


In [ ]:
# Load in the second dataset, which is referred as Dataset 1
dataset = load_dataset("UWV/veringewikkelderingen")
# remove unnecessary columns
dataset = dataset.remove_columns(['jargon', 'official', 'archaïsche_taal', 'technical', 'academic', 'poetic'])
train_dataset = dataset["train"]
train_df = train_dataset.to_pandas()
# split the data, retaining 20% for validation
train_data, eval_data = train_test_split(train_df, test_size=0.2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/1490 [00:00<?, ? examples/s]

In [ ]:
# Merge the first and the synthetic dataset to create Dataset 2
merged_df = pd.concat([train_df, df], ignore_index=True)
merged_df
train_data2, eval_data2 = train_test_split(merged_df, test_size=0.2)

##Function for both models

In [ ]:
# Function for computing Bleu scores for the training and validation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Tokenizing the predictions and references
    tokenized_preds = [nltk.word_tokenize(pred) for pred in decoded_preds]
    tokenized_refs = [[nltk.word_tokenize(ref)] for ref in decoded_labels]

    # Computing BLEU score using NLTK's corpus_bleu
    bleu_score = corpus_bleu(tokenized_refs, tokenized_preds)
    return {"bleu": bleu_score}

In [ ]:
# Function for tokenizing the data
def tokenize_function(examples):
    inputs = [text for text in examples["original"]]
    targets = examples["simplified"]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, padding="max_length", truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Function for simplifying text after training
def simplify_text(text):
    device = next(model.parameters()).device
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Generate the simplified text
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=512,
        num_beams=2,
        early_stopping=True
    )
    # Decode the simplified text
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return simplified_text

##Training for MBart

In [ ]:
#Load in the model and tokenizer for the Dutch-to-Dutch version of MBart
model_name = "facebook/mbart-large-50-many-to-one-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="nl_XX", tgt_lang="nl_XX")

### Tokenizing datasets

In [ ]:
#Tokenize train and test dataset for Dataset 1
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(eval_data)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1192 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/298 [00:00<?, ? examples/s]

In [ ]:
#Tokenize train and test dataset for Dataset 2
train_dataset2 = Dataset.from_dict(train_data2)
test_dataset2 = Dataset.from_dict(eval_data2)

train_dataset2 = train_dataset2.map(tokenize_function, batched=True)
test_dataset2 = test_dataset2.map(tokenize_function, batched=True)

Map:   0%|          | 0/1312 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

### Training and results for dataset 1

In [ ]:
# Set the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)
# Build the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.485200,0.446097,0.126111
2,0.340700,0.423507,0.140589


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'num_beams': 5, 'forced_bos_token_id': 250004, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'num_beams': 5, 'forced_bos_token_id': 250004, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_

TrainOutput(global_step=2384, training_loss=0.5971627011395141, metrics={'train_runtime': 3438.3858, 'train_samples_per_second': 0.693, 'train_steps_per_second': 0.693, 'total_flos': 2583218683379712.0, 'train_loss': 0.5971627011395141, 'epoch': 2.0})

In [ ]:
# Simplify sentence 1 for Mbart-Dataset 1
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Er is een groep mensen die zich kunnen helpen met hun werk. Ze hebben een speciale prijsje, dat is, 20 Euro. Maar I wilden dat ze wel 90 % van deze prijsje terug kon geven. In de lijst die erin zit, kun je het geld verplenden zien. Als je het geld overdoes, wilt je het geld weg om te betalen.


In [ ]:
# Simplify sentence 2 for Mbart-Dataset 1
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de eerste tender van 2022 is er een speciale prijs voor een project. De mensen die me overleed, vertegenwoordigen me in het team. I congratulate u daarof en wish u veel succes met het project.


In [ ]:
# Simplify sentence 3 for Mbart-Dataset 1
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: We hebben geen aanvraag voor een transfer gemaakt. Er is ook geen request dat we de periode voor de transfer kunnen veranderen. Als dat gebeurt, gaat het geld weg.


In [ ]:
# Simplify sentence 4 for Mbart-Dataset 1
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: U zorgt annually voor de progress van mensen die zich laten laten zien hoe het project progresst. Deze taak is binnen de dag dat het begin date van het project is, wat betekent dat het niet meer meer geld is om te doen. Dit betekent dat het project niet meer geld is om te doen.


###Training and results for dataset 2

In [ ]:
# Set the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.008,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
)
# Build the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset2,
    eval_dataset=test_dataset2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.442400,0.411710,0.147337
2,0.312300,0.388534,0.158312


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'num_beams': 5, 'forced_bos_token_id': 250004, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'num_beams': 5, 'forced_bos_token_id': 250004, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_

TrainOutput(global_step=2624, training_loss=0.5468288340219637, metrics={'train_runtime': 3848.3984, 'train_samples_per_second': 0.682, 'train_steps_per_second': 0.682, 'total_flos': 2843274255532032.0, 'train_loss': 0.5468288340219637, 'epoch': 2.0})

In [ ]:
# Simplify sentence 1 for Mbart-Dataset 2
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Naar uw aanvraag, do I het besluit om dit verzoek te wijzen. De subsidieverlening is binnen de limit van €20. Voorschotten, wat voorschotten zijn, hebben I besloten om tot 90% van de subsidie te geven. In de lijst staat er een nieuwschema met overzicht van de voorshotbedragen. Het restantende bedrag zal I nekenen met de subsidievaststelling.


In [ ]:
# Simplify sentence 2 for Mbart-Dataset 2
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de eerste tender van 2022 is een project heel belangrijk. I congratulate u daar en wish u veel succes met het uitvoering.


In [ ]:
# Simplify sentence 3 for Mbart-Dataset 2
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: We hebben geen aanvraag voor overdracht ontvangen. Ook hebben we geen request tot de periode voor overdracht uitsteld. Dit betekent dat het voucher van rechtswege verandert.


In [ ]:
# Simplify sentence 4 for Mbart-Dataset 2
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: U rapporteerd ervoor dat de activiteiten binnen 6 weken na afloop van elk kalenderjaar begonnen.


##Training for T5

In [ ]:
# Load in the model and the tokenizer for the Dutch version of the T5 model
model_name = "yhavinga/t5-base-dutch"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

### tokenizing datasets

In [ ]:
# Tokenize Dataset 1 for the T5 model
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(eval_data)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1192 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/298 [00:00<?, ? examples/s]

In [ ]:
# Tokenize Dataset 2 for the T5 model
train_dataset2 = Dataset.from_dict(train_data2)
test_dataset2 = Dataset.from_dict(eval_data2)

train_dataset2 = train_dataset2.map(tokenize_function, batched=True)
test_dataset2 = test_dataset2.map(tokenize_function, batched=True)

Map:   0%|          | 0/1312 [00:00<?, ? examples/s]

Map:   0%|          | 0/328 [00:00<?, ? examples/s]

###Training and results for dataset 1

In [ ]:
# Set the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=8e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
)
# Build the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.354300,0.357796,0.003320
2,0.247900,0.335037,0.003199


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2384, training_loss=0.7790165512553797, metrics={'train_runtime': 1305.7919, 'train_samples_per_second': 1.826, 'train_steps_per_second': 1.826, 'total_flos': 1451755611095040.0, 'train_loss': 0.7790165512553797, 'epoch': 2.0})

In [ ]:
# Simplify sentence 1 for T5-Dataset 1
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de nieuwe wet is het belangrijk dat iedereen weet wat er gebeurt als je subsidie wilt ontvangen. Dit is belangrijk omdat de overheid regels heeft gemaakt om te zorgen dat de subsidie goed wordt gebruikt. Dit is belangrijk omdat de overheid regels heeft gemaakt om de subsidie te gebruiken.


In [ ]:
# Simplify sentence 2 for T5-Dataset 1
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de nieuwe wet is het belangrijk dat iedereen weet wat er gebeurt als er subsidie wordt verstrekt. Dit is belangrijk omdat de overheid regels heeft gemaakt om te zorgen dat de subsidie goed wordt gebruikt. Dit is belangrijk omdat de overheid regels heeft gemaakt om te zorgen dat de subsidie goed wordt gebruikt.


In [ ]:
# Simplify sentence 3 for T5-Dataset 1
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de nieuwe wet is het belangrijk dat iedereen weet wat er gebeurt als je subsidie wilt ontvangen. Dit is belangrijk omdat de overheid regels heeft gemaakt om te zorgen dat de regels die de overheid heeft gemaakt, goed zijn en dat de regels die ervoor zorgen dat mensen die subsidie willen aanvragen, goed kunnen profiteren van de regelingen die de overheid heeft afgesproken.


In [ ]:
# Simplify sentence 4 for T5-Dataset 1
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de nieuwe wet is het belangrijk dat iedereen weet wat er gebeurt als je subsidie wilt ontvangen. Dit is belangrijk omdat iedereen weet wat er gebeurt als je subsidie wilt.


###Training and results for dataset 2

In [ ]:
# Set the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=8e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
)
# Build the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset2,
    eval_dataset=test_dataset2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,0.373300,0.328229,0.004342
2,0.312900,0.310520,0.004492


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2624, training_loss=0.7148708226840671, metrics={'train_runtime': 1386.3885, 'train_samples_per_second': 1.893, 'train_steps_per_second': 1.893, 'total_flos': 1597905504829440.0, 'train_loss': 0.7148708226840671, 'epoch': 2.0})

In [ ]:
# Simplify sentence 1 for T5-Dataset 2
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In uw aanvraag heb ik besloten om voorschotten te verstrekken tot 90% van de verleende subsidie. Ik heb het besluit genomen om het verzoek toe te wijzen. Ik heb het besluit genomen om het verzoek toe te wijzen. Ik heb besloten om voorschotten te verstrekken tot 90% van de verleende subsidie. Het resterende bedrag zal ik met de subsidievaststelling verrekenen.


In [ ]:
# Simplify sentence 2 for T5-Dataset 2
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In de eerste tender van 2022 heeft uw project de hoogste score gekregen.


In [ ]:
# Simplify sentence 3 for T5-Dataset 2
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: In het verleden hebben we geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht.


In [ ]:
# Simplify sentence 4 for T5-Dataset 2
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: De voortgang van de projecten is 6 weken na de start van het project.


#Zero and few shot models

The following block pertains to the zero-shot and few-shot prompting strategies on 4 different models, the Dutch versions of the T5, MBart, GPT2 and Phi-2 model

## zero shot

In [ ]:
# Function for simplifying text without training
def simplify_text(text, max_length=500):

    prompt = f"""
        Ik wil dat je mijn complexe zin vervangt met een simpele zin.
        De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
        Complex: {text}
        Simpel:
        """
    # Make sure the whole code is either run on CPU or GPU netirely
    device = next(model.parameters()).device
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    text_tokenized = tokenizer.encode(text, return_tensors="pt")

    # Define the dynamic max length
    input_length = len(inputs[0])
    text_length = len(text_tokenized[0])
    max_length = int(input_length + text_length)

    # Generate the simplified text
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode the output
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the simplified text from the generated text
    simplified_text = simplified_text.replace(prompt, "").strip()

    return simplified_text

###Mbart

In [ ]:
# Load in the model and the tokenizer for the MBart model
model_name = "facebook/mbart-large-50-many-to-one-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="nl_XX", tgt_lang="nl_XX")

In [ ]:
# Simplify sentence 1 for MBart-zero-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: I'd graag dat je een simpele zin kan gebruiken als een replacement van een complexe zin. De meaning van deze zin moet het hetzelfde blijven, maar je kunt het makkelijker maken. Complexe zin: Na het geld dat je meende, do I affirm dat je het geld accepted heeft. De geld die je meende, kun je niet meer dan €20 betalen. I've besloten om voorschotten te verstrekken tot 90% van de geld die je meende. In de lijst die er zit, kun je het overzicht van de voorshotbedragen vinden. Het geld die je nog won't betalen, zal je omdat je de geld wegdoet. Simpel:


In [ ]:
# Simplify sentence 2 for MBart-zero-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: I'd graag dat je een simpele zin gebruikte instead van een complexe zin. De meaning van een zin moet het hetzelfde blijven, maar maakt het makkelijker. Complexe: In de eerste tender voor het jaar 2022 is het meest belangrijk voor een project. I congratulate u en wish u veel succes met het ontwerp. Simpel:


In [ ]:
# Simplify sentence 3 for MBart-zero-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: I'd graag dat je een simpele zin kan gebruiken als een replacement van een complexe zin. De meaning van deze zin moet het hetzelfde blijven, maar je kunt het makkelijker maken. Complexe zin: We hebben geen aanvraag voor overdracht gekregen en ook geen verzoek tot de periode voor overdracht laten komen. Als dat gebeurt, wilt je de voucher van rechtswege niet meer gebruikt worden. Simpel:


In [ ]:
# Simplify sentence 4 for MBart-zero-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: I'd graag dat je een simpele zin gebruikte instead van een complexe zin. De betekenis van deze zin moet het hetzelfde blijven, maar je kunt het makkelijker maken. Complexe zin: Every jaar, na het einde van elk jaar van het jaar, kun je de progress van de activities binnen de groep van 6 weken laten zien. Dit betekent dat de start date van het project is. Simpel:


###T5

In [ ]:
# Load in the model and the tokenizer for the T5 model
model_name = "yhavinga/t5-base-dutch"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
# Simplify sentence 1 for T5-zero-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: simpele zin. Simpel:. Simpel: simpele zin. Complex: simpele zin. Simpel:. Complex: simpele zin. Complex: u een complexe zin. Complex:. Complex:. Complex: Complex:Eenvoudig:. Complex: simpele zin. Complex:


In [ ]:
# Simplify sentence 2 for T5-zero-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: simpele zin. Simpel: uw project is in de eerste tender van 2022 als hoogste gerangschikt. Simpel: uw project is in de eerste tender als hoogste gerangschikt. Complex:. Simpel: uw project is afgerond. Complex   in de eerste tenderuw project is als hoogste gerangschikt. Simpel: uw project is in de eerste tender als hoogste gerangschikt.uw project is in de eerste tender van 2022 als


In [ ]:
# Simplify sentence 3 for T5-zero-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: . Complex: We hebben geen aanvraag voor overdracht ontvangen en ook geen aanvraag voor overdracht ontvangen. Complex: We hebben geen aanvraag voor overdracht ontvangen en ook geen aanvraag voor overdracht ontvangen. Complex:. Complex: We hebben geen aanvraag voor overdracht ontvangen. Complex: uw voucher . Complex: We hebben geen aanvraag ontvangenEenvoudig: We hebben geen aanvraag voor overdracht ontvangen. Complex:Bedankt voor uw reactie. Complex: We hebben geen aanvraag voor overdracht ontvangen en ook


In [ ]:
# Simplify sentence 4 for T5-zero-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: simpele zin. Complex: U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project. Complex: U rapporteert jaarlijks over de voortgang van de activiteiten. Complex: Complex:. Complex: U rapporteert jaarlijks over de voortgang  Complexu rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de start


###GPT2

In [ ]:
# Load in the model and the tokenizer for the GPT2 model
model_name = "GroNLP/gpt2-small-dutch"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/108 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/652k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/382k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

In [ ]:
# Simplify sentence 1 for GPT2-zero-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 
        Ik wil dat je mijn complexe zin vervangt met een simpele zin.
        De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
        Complex: Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. 
        Simpel:
          Ik wil dat je mijn complexe zin vervangt met een simpele zin.
             De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
             Complex: Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voor


In [ ]:
# Simplify sentence 2 for GPT2-zero-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 
        Ik wil dat je mijn complexe zin vervangt met een simpele zin.
        De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
        Complex: Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering.
        Simpel:
                                                                            


In [ ]:
# Simplify sentence 3 for GPT2-zero-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


In [ ]:
# Simplify sentence 4 for GPT2-zero-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


###Fietje-2

In [ ]:
# Load in the model and the tokeizer for Fietje-2
model_name = "BramVanroy/fietje-2"
model = PhiForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Simplify sentence 1 for Fietje-2 - zero-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 2 for Fietje-2 - zero-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 3 for Fietje-2 - zero-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 4 for Fietje-2 - zero-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

## few shot

The following blocks pertain to the fewshot prompting approach for all four models

### compression rate

In [ ]:
# Function for calculating the compression rate for the synthetic dataset
def compression_ratio(row):
    c_length = len(row['original'])
    ri_length = len(row['simplified'])
    return c_length / ri_length if ri_length != 0 else float('inf')

df['compression_ratio'] = df.apply(compression_ratio, axis=1)

In [ ]:
# Select the top two for the examples in the few-shot prompting
sorted_df = df.sort_values(by='compression_ratio', ascending=False)

top_2_df = sorted_df.head(3)

top_2_df

,original,simplified,compression_ratio
9,Ten aanzien van de naleving van de nieuwe wetg...,Voor de naleving van de nieuwe wet is het bela...,1.592000
0,"De overheid heeft besloten dat, in het kader v...",De overheid heeft nieuwe regels voor subsidies...,1.582011
2,In het geval dat de subsidieaanvrager niet bin...,Als de aanvrager niet op tijd aan alle voorwaa...,1.577181


###Simplify function

In [ ]:
# Function to simplify text with few-shot prompting
def simplify_text(text, max_length=500):

    prompt = f"""
        Ik wil dat je mijn complexe zin vervangt met een simpele zin.
        De betekenis van de zin moet hetzelfde blijven, maar maak het makkelijker.
        Complex: Ten aanzien van de naleving van de nieuwe wetgeving is het van belang dat alle betrokken partijen op de hoogte zijn van de specifieke verplichtingen en verantwoordelijkheden die hieruit voortvloeien.
        Simpel: Voor de naleving van de nieuwe wet is het belangrijk dat iedereen weet welke verplichtingen en verantwoordelijkheden er zijn.
        Complex: De overheid heeft besloten dat, in het kader van de nieuwe subsidieregeling, organisaties die in aanmerking willen komen voor financiële ondersteuning, moeten voldoen aan een reeks strengere criteria, welke zijn opgesteld om de efficiëntie en effectiviteit van de verstrekte subsidies te waarborgen.
        Simpel: De overheid heeft nieuwe regels voor subsidies. Organisaties die subsidies willen, moeten nu aan strengere criteria voldoen. Deze criteria zorgen ervoor dat subsidies beter worden gebruikt.
        Complex: {text}
        Simpel:
        """
    # Make sure the code runs entirely on CPU or GPU
    device = next(model.parameters()).device
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    text_tokenized = tokenizer.encode(text, return_tensors="pt")

    # Set the dynamic max length
    input_length = len(inputs[0])
    text_length = len(text_tokenized[0])
    max_length = int(input_length + text_length)

    # Generate the simplified text
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )

    # Decode the output
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the simplified text from the generated text
    simplified_text = simplified_text.replace(prompt, "").strip()

    return simplified_text

### Mbart

In [ ]:
# Load in the model and the tokenizer for the MBart model
model_name = "facebook/mbart-large-50-many-to-one-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="nl_XX", tgt_lang="nl_XX")

In [ ]:
# Simplify sentence 1 for MBart-few-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben gemaakt. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat deze regels hebben g

In [ ]:
# Simplify sentence 2 for MBart-few-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, weten wat deze regels hebben. Het is heel belangrijk dat alle mensen die in de regering willen doen, wet

In [ ]:
# Simplify sentence 3 for MBart-few-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten wat obligations en verantwoordelijkheden er zijn. Het is heel belangrijk dat alle mensen die in de regering willen werken, weten w

In [ ]:
# Simplify sentence 4 for MBart-few-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze moeten doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel belangrijk dat alle mensen die in het project willen doen, weten wat ze kunnen doen. Het is heel b

###T5

In [ ]:
# Load in the model and the tokenizer for the T5 model
model_name = "yhavinga/t5-base-dutch"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

In [ ]:
# Simplify sentence 1 for T5-few-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: . Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies.. Complex: De overheid heeft nieuwe regels voor subsidies.. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: de overheid............... Complex:... Complex:. Complex:. Complex:. Complex:. Complex:. Complex:Omdat


In [ ]:
# Simplify sentence 2 for T5-few-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: en verantwoordelijkheden. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies.. Complex:. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: de overheid...   Uw project is als hoogste gerangschikt. Complex: Uw project is als hoogste gerangschikt.Bedankt voor uw inzet. Complex: Uw project is in de eerste tender van 2022 als


In [ ]:
# Simplify sentence 3 for T5-few-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: . Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies... Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: de overheid...Bedankt voor uw voucher. Complex: De overheid heeft besloten dat organisaties die in aanmerking willen komen voor financiële ondersteuning, moeten voldoen aan een reeks strengere criteria. Complex: De overheid heeft besloten


In [ ]:
# Simplify sentence 4 for T5-few-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: . Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies.. Complex: nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: De overheid heeft nieuwe regels voor subsidies. Complex: de nieuwe wet.. Complex.u rapporteert jaarlijks over de voortgang van de activiteiten. Complex: U rapporteert jaarlijks over de voortgangJaarlijks rapporteert u over de voortgang van de activiteiten. Complex: U rapporteert jaarlijks over de voortgang van de activiteiten


###GPT2

In [ ]:
# Load in the model and the tokenizer for the GPT2 model
model_name = "GroNLP/gpt2-small-dutch"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Simplify sentence 1 for GPT2-few-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


In [ ]:
# Simplify sentence 2 for GPT2-few-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


In [ ]:
# Simplify sentence 3 for GPT2-few-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


In [ ]:
# Simplify sentence 4 for GPT2-few-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

Simplified text: 


###Fietje

In [ ]:
# Load in the model and the tokenizer for the Fietje-2 model
model_name = "BramVanroy/fietje-2"
model = PhiForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Simplify sentence 1 for Fietje-2 - few-shot prompting
original_text = "Naar aanleiding van uw aanvraag, heb ik het besluit genomen om dit verzoek toe te wijzen. De subsidieverlening bedraagt maximaal €20. Ik heb ambtshalve besloten voorschotten te verstrekken tot 90% van de verleende subsidie. In de bijlage is het betaalschema met overzicht van de voorshotbedragen te vinden. Het resterende bedrag zal ik met de subsidievaststelling verrekenen. "
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 2 for Fietje-2 - few-shot prompting
original_text = "Uw project is in de eerste tender van 2022 als hoogste gerangschikt. Mede namens mijn collega's feliciteer ik u hiermee en wens ik u veel succes met de uitvoering."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 3 for Fietje-2 - few-shot prompting
original_text = "We hebben geen aanvraag voor overdracht ontvangen en ook geen verzoek tot uitstel van de periode voor overdracht. Hierdoor vervalt uw voucher van rechtswege."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")

In [ ]:
# Simplify sentence 4 for Fietje-2 - few-shot prompting
original_text = "U rapporteert jaarlijks over de voortgang van de activiteiten binnen 6 weken na afloop van elk kalenderjaar gerekend vanaf de startdatum van het project."
simplified_text = simplify_text(original_text)
print(f"Simplified text: {simplified_text}")